# Data
In this specfi task i decided to use maritl, jobclass, health and health_ins to predict wage. All of this features will be transformed to dummy variables (like 0/1)

In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ISLP import load_data
from sklearn.model_selection import train_test_split

data = load_data('Wage')


X, y = data[['maritl', 'jobclass', 'health', 'health_ins']], data[['wage']]

X

,maritl,jobclass,health,health_ins
0,1. Never Married,1. Industrial,1. <=Good,2. No
1,1. Never Married,2. Information,2. >=Very Good,2. No
2,2. Married,1. Industrial,1. <=Good,1. Yes
3,2. Married,2. Information,2. >=Very Good,1. Yes
4,4. Divorced,2. Information,1. <=Good,1. Yes
...,...,...,...,...
2995,2. Married,1. Industrial,2. >=Very Good,1. Yes
2996,2. Married,1. Industrial,2. >=Very Good,2. No
2997,2. Married,1. Industrial,1. <=Good,2. No
2998,1. Never Married,1. Industrial,2. >=Very Good,1. Yes


## Nominal Values to numbers


In [27]:
def transform_x(row):
    Result = {}

    #Merrige
    Result['merried'] = 1 if row['maritl'] == '2. Married' or row['maritl'] == '5. Separated' else 0
    Result['divorced'] = 1 if row['maritl'] == '3. Divorced' else 0
    Result['Separated'] = 1 if row['maritl'] == '5. Separated' else 0

    #job class
    Result['indrustial'] = 1 if row['jobclass'] == '1. Industrial' else 0

    #health & ins
    Result['health'] = 1 if row['health'] == '2. >=Very Good' else 0
    Result['health_ins'] = 1 if row['health_ins'] == '1. Yes' else 0

    return pd.Series(Result)


X_num = X.apply(transform_x, axis=1)

X_num

,merried,divorced,Separated,indrustial,health,health_ins
0,0,0,0,1,0,0
1,0,0,0,0,1,0
2,1,0,0,1,0,1
3,1,0,0,0,1,1
4,0,0,0,0,0,1
...,...,...,...,...,...,...
2995,1,0,0,1,1,1
2996,1,0,0,1,1,0
2997,1,0,0,1,0,0
2998,0,0,0,1,1,1


## Data Split

In [28]:
x_train, x_test, y_train, y_test = train_test_split(X_num, y, test_size=0.2, random_state=42)

x_train

,merried,divorced,Separated,indrustial,health,health_ins
642,0,0,0,0,0,1
700,0,0,0,1,1,1
226,0,0,0,1,1,0
1697,1,0,0,0,1,1
1010,1,0,0,0,1,0
...,...,...,...,...,...,...
1638,1,0,0,0,1,1
1095,1,0,0,1,1,1
1130,1,0,0,0,1,0
1294,1,0,0,0,1,1


## Data Vizualization

## Scatter matrix